In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
import lightgbm as lgb
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, PolynomialFeatures
from mlxtend.feature_selection import SequentialFeatureSelector as SFS

import warnings
warnings.filterwarnings("ignore")

In [2]:
# To display all rows
pd.set_option('display.max_rows', None)

# To display all columns
pd.set_option('display.max_columns', None)

# To display entire contents of each cell (useful for long strings)
pd.set_option('display.max_colwidth', None)

In [3]:
# Load the train and test datasets
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

df_train.columns

Index(['id', 'Sex', 'Length', 'Diameter', 'Height', 'Weight', 'Shucked Weight',
       'Viscera Weight', 'Shell Weight', 'Age'],
      dtype='object')

In [4]:
df_test.columns

Index(['id', 'Sex', 'Length', 'Diameter', 'Height', 'Weight', 'Shucked Weight',
       'Viscera Weight', 'Shell Weight'],
      dtype='object')

In [5]:
# Separate features and target for train and validation data

X_train = df_train.drop(["id", "Age"], axis=1)
y_train = df_train["Age"]

X_test = df_test.drop(["id"], axis=1)

In [6]:
label_encoder = LabelEncoder()
X_train["Sex"] = label_encoder.fit_transform(X_train["Sex"])
X_test["Sex"] = label_encoder.transform(X_test["Sex"])

In [7]:
X_train.head()

,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight
0,1,1.5250,1.1750,0.3750,28.973189,12.728926,6.647958,8.348928
1,1,1.1000,0.8250,0.2750,10.418441,4.521745,2.324659,3.401940
2,2,1.3875,1.1125,0.3750,24.777463,11.339800,5.556502,6.662133
3,0,1.7000,1.4125,0.5000,50.660556,20.354941,10.991839,14.996885
4,1,1.2500,1.0125,0.3375,23.289114,11.977664,4.507570,5.953395


In [8]:
y_train.head()

0     9
1     8
2     9
3    11
4     8
Name: Age, dtype: int64

In [9]:
X_test.head()

,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight
0,1,1.0500,0.7625,0.2750,8.618248,3.657085,1.729319,2.721552
1,1,1.1625,0.8875,0.2750,15.507176,7.030676,3.246018,3.968930
2,0,1.2875,0.9875,0.3250,14.571643,5.556502,3.883882,4.819415
3,0,1.5500,0.9875,0.3875,28.377849,13.380964,6.548735,7.030676
4,1,1.1125,0.8500,0.2625,11.765042,5.528153,2.466407,3.331066


In [10]:
def generate_interactive_features(df, df_features):
    """
    Generate interaction features between the given columns in a DataFrame.

    Parameters:
    -----------
    df : pandas.DataFrame
        The DataFrame containing the original features.
    df_features : list
        A list of feature names to be used for generating interaction features.

    Returns:
    --------
    pandas.DataFrame
        The DataFrame with the interaction features only.
    """
    # Get the list of features to create interaction terms
    features = [col for col in df.columns if col in df_features]
    new_features = []

    # Create a new DataFrame to store only the new features
    df_new_features = pd.DataFrame(index=df.index)

    # Iterate through the features and create interaction terms
    for i in range(len(features)):
        for j in range(i+1, len(features)):
            if i != j:
                # Generate a new feature name for the interaction term
                new_feature_name = f"{features[i]}_{features[j]}"
                
                # Create the interaction feature by multiplying the values of the two original features
                df_new_features[new_feature_name] = np.where(df[features[i]] == 0, 1e-10, df[features[i]]) * np.where(df[features[j]] == 0, 1e-10, df[features[j]])
                
                # Add the new feature name to the list of new features
                new_features.append(new_feature_name)
    
    return df_new_features

In [11]:
def generate_triple_interactive_features(df, df_features):
    """
    Generate interaction features between three of the given columns in a DataFrame.

    Parameters:
    -----------
    df : pandas.DataFrame
        The DataFrame containing the original features.
    df_features : list
        A list of feature names to be used for generating interaction features.

    Returns:
    --------
    pandas.DataFrame
        The DataFrame with the interaction features added.
    """
    # Get the list of features to create interaction terms
    features = [col for col in df.columns if col in df_features]
    new_features = []

    # Create a new DataFrame to store only the new features
    df_new_features = pd.DataFrame(index=df.index)

    # Iterate through the features and create interaction terms
    for i in range(len(features)):
        for j in range(i+2, len(features)):
            for k in range(i+3, len(features)):
                if i != j and i != k and j != k:
                    # Generate a new feature name for the interaction term
                    new_feature_name = f"{features[i]}_{features[j]}_{features[k]}"
                    
                    # Create the interaction feature by multiplying the values of the two original features
                    df_new_features[new_feature_name] = np.where(df[features[i]] == 0, 1e-10, df[features[i]]) * np.where(df[features[j]] == 0, 1e-10, df[features[j]]) * np.where(df[features[k]] == 0, 1e-10, df[features[k]])
                    
                    # Add the new feature name to the list of new features
                    new_features.append(new_feature_name)
    
    return df_new_features

In [12]:
def generate_polynomial_features(df, degree, df_features):
    """
    Generate polynomial features for the specified columns in a DataFrame.

    Parameters:
    -----------
    df : pandas.DataFrame
        The DataFrame containing the original features.
    degree : int
        The degree of the polynomial features to generate.
    df_features : list
        A list of feature names to be used for generating polynomial features.

    Returns:
    --------
    pandas.DataFrame
        The DataFrame with the polynomial features added.
    """
    # Get the list of features to create polynomial features
    features = [col for col in df.columns if col in df_features]

    # Create a PolynomialFeatures object with the specified degree, no interaction features, and no bias term
    poly = PolynomialFeatures(degree, interaction_only=False, include_bias=False)

    # Fit and transform the selected features in the DataFrame
    poly_features = poly.fit_transform(df[features])

    # Get the feature names for the generated polynomial features
    poly_features_names = poly.get_feature_names_out(features)

    # Create a new DataFrame with the generated polynomial features
    poly_df = pd.DataFrame(poly_features, columns=poly_features_names)

    # Keep only the columns with polynomial features of the specified degree
    poly_df = poly_df[[f"{col}^{degree}" for col in features]]

    # # Concatenate the original DataFrame and the polynomial features DataFrame
    # result = pd.concat([df, poly_df], axis=1)

    return poly_df

In [13]:
def generate_domain_features(df, df_features):
    """
    Generate domain-specific features as ratios between the given columns in a DataFrame.

    Parameters:
    -----------
    df : pandas.DataFrame
        The DataFrame containing the original features.
    df_features : list
        A list of feature names to be used for generating domain-specific features.

    Returns:
    --------
    pandas.DataFrame
        The DataFrame with the domain-specific features added.
    """
    # Get the list of features to create domain-specific features
    features = [col for col in df.columns if col in df_features]
    new_features = []

    # Create a new DataFrame to store only the new features
    df_new_features = pd.DataFrame(index=df.index)

    # Iterate through the features and create domain-specific features as ratios
    for i in range(len(features)):
        for j in range(len(features)):
            # Check if the features are different
            if i != j:
                # Generate a new feature name for the domain-specific feature
                new_feature_name = f"{features[i]}_{features[j]}_ratio"
                
                # Create the domain-specific feature by dividing the values of the two original features
                # If the denominator is 0, use a small value (1e-6) to avoid division by zero
                df_new_features[new_feature_name] = np.where(df[features[i]] == 0, 1e-10, df[features[i]]) / np.where(df[features[j]] == 0, 1e-10, df[features[j]])
                
                # Add the new feature name to the list of new features
                new_features.append(new_feature_name)
    
    return df_new_features

In [14]:
# Generate features for the training dataset
inter_features = generate_interactive_features(X_train, X_train.columns)
triple_inter_features = generate_triple_interactive_features(X_train, X_train.columns)
poly_features = generate_polynomial_features(X_train, 2, X_train.columns)
ratio_features = generate_domain_features(X_train, X_train.columns)
X_train = pd.concat([X_train, inter_features, triple_inter_features, poly_features, ratio_features], axis=1)
X_train.shape

(74051, 155)

In [15]:
# Generate features for the training dataset
inter_features = generate_interactive_features(X_test, X_test.columns)
triple_inter_features = generate_triple_interactive_features(X_test, X_test.columns)
poly_features = generate_polynomial_features(X_test, 2, X_test.columns)
ratio_features = generate_domain_features(X_test, X_test.columns)
X_test = pd.concat([X_test, inter_features, triple_inter_features, poly_features, ratio_features], axis=1)
X_test.columns

Index(['Sex', 'Length', 'Diameter', 'Height', 'Weight', 'Shucked Weight',
       'Viscera Weight', 'Shell Weight', 'Sex_Length', 'Sex_Diameter',
       ...
       'Viscera Weight_Weight_ratio', 'Viscera Weight_Shucked Weight_ratio',
       'Viscera Weight_Shell Weight_ratio', 'Shell Weight_Sex_ratio',
       'Shell Weight_Length_ratio', 'Shell Weight_Diameter_ratio',
       'Shell Weight_Height_ratio', 'Shell Weight_Weight_ratio',
       'Shell Weight_Shucked Weight_ratio',
       'Shell Weight_Viscera Weight_ratio'],
      dtype='object', length=155)

In [16]:
X_test.head()

,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Sex_Length,Sex_Diameter,Sex_Height,Sex_Weight,Sex_Shucked Weight,Sex_Viscera Weight,Sex_Shell Weight,Length_Diameter,Length_Height,Length_Weight,Length_Shucked Weight,Length_Viscera Weight,Length_Shell Weight,Diameter_Height,Diameter_Weight,Diameter_Shucked Weight,Diameter_Viscera Weight,Diameter_Shell Weight,Height_Weight,Height_Shucked Weight,Height_Viscera Weight,Height_Shell Weight,Weight_Shucked Weight,Weight_Viscera Weight,Weight_Shell Weight,Shucked Weight_Viscera Weight,Shucked Weight_Shell Weight,Viscera Weight_Shell Weight,Sex_Diameter_Height,Sex_Diameter_Weight,Sex_Diameter_Shucked Weight,Sex_Diameter_Viscera Weight,Sex_Diameter_Shell Weight,Sex_Height_Weight,Sex_Height_Shucked Weight,Sex_Height_Viscera Weight,Sex_Height_Shell Weight,Sex_Weight_Height,Sex_Weight_Shucked Weight,Sex_Weight_Viscera Weight,Sex_Weight_Shell Weight,Sex_Shucked Weight_Height,Sex_Shucked Weight_Weight,Sex_Shucked Weight_Viscera Weight,Sex_Shucked Weight_Shell Weight,Sex_Viscera Weight_Height,Sex_Viscera Weight_Weight,Sex_Viscera Weight_Shucked Weight,Sex_Viscera Weight_Shell Weight,Sex_Shell Weight_Height,Sex_Shell Weight_Weight,Sex_Shell Weight_Shucked Weight,Sex_Shell Weight_Viscera Weight,Length_Height_Weight,Length_Height_Shucked Weight,Length_Height_Viscera Weight,Length_Height_Shell Weight,Length_Weight_Shucked Weight,Length_Weight_Viscera Weight,Length_Weight_Shell Weight,Length_Shucked Weight_Weight,Length_Shucked Weight_Viscera Weight,Length_Shucked Weight_Shell Weight,Length_Viscera Weight_Weight,Length_Viscera Weight_Shucked Weight,Length_Viscera Weight_Shell Weight,Length_Shell Weight_Weight,Length_Shell Weight_Shucked Weight,Length_Shell Weight_Viscera Weight,Diameter_Weight_Shucked Weight,Diameter_Weight_Viscera Weight,Diameter_Weight_Shell Weight,Diameter_Shucked Weight_Viscera Weight,Diameter_Shucked Weight_Shell Weight,Diameter_Viscera Weight_Shucked Weight,Diameter_Viscera Weight_Shell Weight,Diameter_Shell Weight_Shucked Weight,Diameter_Shell Weight_Viscera Weight,Height_Shucked Weight_Viscera Weight,Height_Shucked Weight_Shell Weight,Height_Viscera Weight_Shell Weight,Height_Shell Weight_Viscera Weight,Weight_Viscera Weight_Shell Weight,Sex^2,Length^2,Diameter^2,Height^2,Weight^2,Shucked Weight^2,Viscera Weight^2,Shell Weight^2,Sex_Length_ratio,Sex_Diameter_ratio,Sex_Height_ratio,Sex_Weight_ratio,Sex_Shucked Weight_ratio,Sex_Viscera Weight_ratio,Sex_Shell Weight_ratio,Length_Sex_ratio,Length_Diameter_ratio,Length_Height_ratio,Length_Weight_ratio,Length_Shucked Weight_ratio,Length_Viscera Weight_ratio,Length_Shell Weight_ratio,Diameter_Sex_ratio,Diameter_Length_ratio,Diameter_Height_ratio,Diameter_Weight_ratio,Diameter_Shucked Weight_ratio,Diameter_Viscera Weight_ratio,Diameter_Shell Weight_ratio,Height_Sex_ratio,Height_Length_ratio,Height_Diameter_ratio,Height_Weight_ratio,Height_Shucked Weight_ratio,Height_Viscera Weight_ratio,Height_Shell Weight_ratio,Weight_Sex_ratio,Weight_Length_ratio,Weight_Diameter_ratio,Weight_Height_ratio,Weight_Shucked Weight_ratio,Weight_Viscera Weight_ratio,Weight_Shell Weight_ratio,Shucked Weight_Sex_ratio,Shucked Weight_Length_ratio,Shucked Weight_Diameter_ratio,Shucked Weight_Height_ratio,Shucked Weight_Weight_ratio,Shucked Weight_Viscera Weight_ratio,Shucked Weight_Shell Weight_ratio,Viscera Weight_Sex_ratio,Viscera Weight_Length_ratio,Viscera Weight_Diameter_ratio,Viscera Weight_Height_ratio,Viscera Weight_Weight_ratio,Viscera Weight_Shucked Weight_ratio,Viscera Weight_Shell Weight_ratio,Shell Weight_Sex_ratio,Shell Weight_Length_ratio,Shell Weight_Diameter_ratio,Shell Weight_Height_ratio,Shell Weight_Weight_ratio,Shell Weight_Shucked Weight_ratio,Shell Weight_Viscera Weight_ratio
0,1,1.0500,0.7625,0.2750,8.618248,3.657085,1.729319,2.721552,1.050000e+00,7.625000e-01,2.750000e-01,8.618248e+00,3.657085e+00,1.729319e+00,2.721552e+00,0.800625,0.288750,9.049160,3.839940,1.815785,2.857630,0.209687,6.571414,2.788528,1

In [17]:
def evaluate_model(model, X, y, n_splits=10):
    """
    Evaluates the given model using cross-validation and calculates the Mean Absolute Errors.

    Parameters:
    -----------
    model (estimator object): The model to be evaluated.
    X (DataFrame): The feature matrix.
    y (Series): The target variable.
    n_splits (int): The number of folds for cross-validation.

    Returns:
    --------
    mae_scores (list): A list of MAE for each fold.
    """
    # Initialize a list to store the MAE
    mae_scores = []

    # Create a KFold object for cross-validation
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=5)

    # Perform cross-validation
    for train_index, test_index in kf.split(X):
        # Split the data into train and test sets for the current fold
        X_train_cv, X_test_cv = X.iloc[train_index], X.iloc[test_index]
        y_train_cv, y_test_cv = y.iloc[train_index], y.iloc[test_index]

        # Train the model on the training data
        model.fit(X_train_cv, y_train_cv)

        # Predict for the test
        y_pred = model.predict(X_test_cv)

        # Round y_pred to the nearest integer
        y_pred = np.round(y_pred).astype(int)

        # Calculate the MAE for the current fold
        mae = mean_absolute_error(y_test_cv, y_pred)
        mae_scores.append(mae)

    # Return the MAE, rounded to 3 decimal places
    return [round(value, 3) for value in mae_scores]

In [18]:
# Initialize the models

models = {
    "LightGBM": lgb.LGBMRegressor(random_state=5),
    # "CatBoost": CatBoostRegressor(silent=True, random_seed=5),
}

In [19]:
# Evaluate each model using the best subset of features
for name, model in models.items():
    """
    Loops through each model, and evaluates the model using cross-validation. 
    Prints the MAE scores, average MAE, and standard deviation
    for each model.
    """
    #Initialize SFS with the current model
    sfs = SFS(model,
              k_features="best",
              forward=True,
              floating=True,
              scoring="neg_mean_absolute_error",
              cv=3,
              n_jobs=-1)
    
    # Perform SFS on the training data
    sfs = sfs.fit(X_train, y_train)

    # Get the selected features
    selected_features = X_train.columns[list(sfs.k_feature_idx_)]

    #Print the results for the current model
    print(f"Model: {name}")
    print(f"Selected features: {selected_features}")

    # Evaluate the model using cross-validation with the selected features
    mae_scores = evaluate_model(model, X_train[selected_features], y_train)
    mean_mae = np.mean(mae_scores)
    std = np.std(mae_scores)

    # Train the model on the training set
    model.fit(X_train[selected_features], y_train)

    # Predict for the test set
    y_test_pred = model.predict(X_test[selected_features])

    df_test["Age"] = np.round(y_test_pred).astype(int)

    # Save the output DataFrame to a CSV file
    df_test[["id", "Age"]].to_csv(f"submission_{name}.csv", index=False)

    # Print the results for the current model
    print(f"Model: {name}")
    print(f"MAE Scores: {mae_scores}")
    print(f"Average MAE: {mean_mae:.3f}")
    print(f"Std Deviation: {std:.3f}")

    # try:
    #     plt.figure(figsize=(10, 7))
    #     plt.plot(model.feature_importances_, label=name)
    #     plt.xticks(np.arange(X_train.shape[1]), X_train.columns.tolist(), rotation=90)
    #     plt.legend()
    
    # except AttributeError: # Incase the model does not have "feature_importances_"
    #     pass
    
    print()


STOPPING EARLY DUE TO KEYBOARD INTERRUPT...

TypeError: 'NoneType' object is not iterable